Alan Andree Rodríguez Levario
222791133

# Hands on 4

## 1.1 Fundamentos de la técnica

La Regresión Logística, a pesar de su nombre, es un algoritmo de clasificación, no de regresión.

Se utiliza para predecir la probabilidad de que una instancia pertenezca a una clase determinada.

A diferencia del Perceptrón simple (que da una salida "dura" de 0 o 1), la Regresión Logística utiliza una función matemática (la función Sigmoide) para transformar la salida en un valor de probabilidad entre 0 y 1.

Si la probabilidad estimada es mayor al 50% ($p \geq 0.5$), se clasifica como clase positiva (1).

Si es menor ($p < 0.5$), se clasifica como clase negativa (0).

Es ampliamente utilizada en medicina (¿tiene el paciente la enfermedad?), finanzas (¿es fraudulenta esta transacción?) y marketing (¿comprará el usuario el producto?).

## 1.2 Modelo Matemático
El corazón de la Regresión Logística es la Función Sigmoide (o logística).

Combinación Lineal ($z$):

Primero, el modelo calcula una suma ponderada de las entradas, similar al Perceptrón:

$$z = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b$$$$z = \mathbf{w} \cdot \mathbf{x} + b$$

Función de Activación (Sigmoide):

Luego, aplicamos la función sigmoide ($\sigma$) a $z$. Esta función "aplasta" cualquier valor real hacia el rango $(0, 1)$.$$h_\theta(x) = \sigma(z) = \frac{1}{1 + e^{-z}}$$

Donde:

$e$ es la base del logaritmo natural (aprox. 2.718).  

$h_\theta(x)$ es la probabilidad estimada de que la salida sea 1.

Decisión:

\hat{y} = \begin{cases}1 & \text{si } h_\theta(x) \ge 0.5 \0 & \text{si } h_\theta(x) < 0.5\end{cases}

## 1.3 Descripción de la Librería

Librería: sklearn.linear_model

* Clase: LogisticRegression

* Descripción: Implementa el algoritmo. Scikit-learn detecta automáticamente si hay más de 2 clases y ajusta la estrategia interna (usualmente "Softmax" o "One-vs-Rest").

* Parámetros clave:

  * max_iter: Número máximo de iteraciones para que el "solucionador" matemático converja. A veces es necesario aumentarlo si los datos no están escalados.

  * solver: El algoritmo matemático para optimizar los pesos (ej. 'lbfgs').


Librería: sklearn.model_selection

* Función: train_test_split

* Descripción: Divide los datos en subconjuntos de entrenamiento y prueba.

Librería: sklearn.preprocessing

* Clase: StandardScaler

* Descripción: La Regresión Logística funciona mucho mejor y converge más rápido si las variables tienen la misma escala (media 0, desviación 1).

Librería: sklearn.metrics

* Funciones: accuracy_score, confusion_matrix.

* Descripción: Para evaluar qué tan bien clasificó el modelo.

## 1.4 Pipeline de Implementación

In [ ]:
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Carga de datos
iris = load_iris()
X = iris.data
y = iris.target

feature_names = iris.feature_names
target_names = iris.target_names

print(f"Características: {feature_names}")
print(f"Clases: {target_names}")

### 1.4.1 Preprocesamiento
#### Análisis Estadístico

In [ ]:
# Crear DataFrame para análisis
df = pd.DataFrame(X, columns=feature_names)
df['species_id'] = y
df['species_name'] = [target_names[i] for i in y]

# Estadísticas descriptivas
print("--- Análisis Estadístico (Describe) ---")
print(df.describe())

#### Gráfico de Dispersión (Scatter Plot)

In [ ]:
# Visualización de datos
print("\n--- Gráfico de Dispersión (Pairplot) ---")
sns.pairplot(df, hue='species_name', markers=["o", "s", "D"])
plt.suptitle("Distribución de Especies Iris", y=1.02)
plt.show()

#### División y Escalado (Feature Engineering)

Aunque la Regresión Logística puede funcionar sin escalar, el algoritmo de optimización interno ('lbfgs') converge mucho más rápido y es más estable con datos escalados.

In [ ]:
# 1. Dividir en Train y Test (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Escalar los datos (StandardScaler)
scaler = StandardScaler()

# Ajustar solo con train
X_train_scaled = scaler.fit_transform(X_train)
# Transformar test
X_test_scaled = scaler.transform(X_test)

#### Entrenamiento del Modelo

In [ ]:
# Instanciar el modelo
# Aumentamos max_iter a 1000 para asegurar convergencia
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Entrenar
log_reg.fit(X_train_scaled, y_train)

### 1.4.2 Prediction
Una gran ventaja de la Regresión Logística es que no solo nos dice la clase, sino la probabilidad de esa clase. He añadido esa información a la función.

In [ ]:
def predecir_especie(patron):
    # 1. Preparar el dato (reshape y escala)
    patron_np = np.array(patron).reshape(1, -1)
    patron_scaled = scaler.transform(patron_np)

    # 2. Predecir la clase
    prediccion_id = log_reg.predict(patron_scaled)[0]
    nombre_especie = target_names[prediccion_id]

    # 3. (Opcional) Ver la probabilidad de confianza
    probabilidades = log_reg.predict_proba(patron_scaled)[0]
    confianza = probabilidades[prediccion_id] * 100

    print(f"Patrón: {patron}")
    print(f"Predicción: {nombre_especie} (Confianza: {confianza:.2f}%)")
    return nombre_especie

# --- Pruebas ---
print("--- Prueba 1: Probable Setosa ---")
predecir_especie([5.1, 3.5, 1.4, 0.2])

print("\n--- Prueba 2: Probable Virginica ---")
predecir_especie([6.7, 3.0, 5.2, 2.3])

### 1.4.3 Model Evaluation

Evaluamos el modelo con el conjunto de prueba.

#### Accuracy

In [ ]:
# Predicciones sobre el Test Set
y_pred = log_reg.predict(X_test_scaled)

# Calcular Accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy:")
print(f"Exactitud del modelo: {accuracy * 100:.2f}%")

#### Matriz de Confusión

In [ ]:
# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens",
            xticklabels=target_names,
            yticklabels=target_names)
plt.ylabel('Verdadero')
plt.xlabel('Predicho')
plt.title('Matriz de Confusión - Regresión Logística')
plt.show()

## 3. Referencias Bibliográficas

Documentación oficial de Scikit-learn: sklearn.linear_model.LogisticRegression. https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
